In [1]:
import numpy as np

import face_recognition

In [2]:
PICS_PATH = 'materials/pictures/'

IMG1 = PICS_PATH+'jb1.png'
IMG2 = PICS_PATH+'jb2.png'

## Face Detection
To 

In [3]:
import cv2

def detect_and_crop_face(image_path):
    # Load Haar Cascade for face detection
    face_cascade = cv2.CascadeClassifier(
        cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'
    )

    # Read the image
    image = cv2.imread(image_path)
    
    # Convert to grayscale
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Detect faces in the image
    faces = face_cascade.detectMultiScale(
        gray_image,
        scaleFactor=1.1,
        minNeighbors=5
    )

    # Check if any faces were found
    if len(faces) == 0:
        print("No faces found.")
        return None

    faces_pics = []
    # Crop and save each detected face
    for (x, y, w, h) in faces:
        face = image[y:y+h, x:x+w]  # Crop the face from the image
        faces_pics.append(face)

    return faces_pics

In [4]:
# Usage example
faces_pics1 = detect_and_crop_face(IMG1)

len(faces_pics1)

1

In [5]:
# Usage example
faces_pics2 = detect_and_crop_face(IMG2)

len(faces_pics2)

2

## Face Encoding
The next step is to extract features from these faces. This can be done using facial embeddings or encodings.

In [6]:
encodings = []

for pic1 in faces_pics1:
    image1_rgb = cv2.cvtColor(pic1, cv2.COLOR_BGR2RGB)
    encoding1 = face_recognition.face_encodings(image1_rgb)
    for pic2 in faces_pics2:
        image2_rgb = cv2.cvtColor(pic2, cv2.COLOR_BGR2RGB)
        encoding2 = face_recognition.face_encodings(image2_rgb)

        if len(encoding1) > 0 and len(encoding2) > 0:
            encodings.append([encoding1[0], encoding2[0]])

len(encodings)

1

In [8]:
encodings[0][0]

array([-0.01751508,  0.07707559, -0.02128401, -0.02624768, -0.0507114 ,
       -0.01067236,  0.01568273, -0.03122935,  0.06684347, -0.08669632,
        0.22173631, -0.04362556, -0.25420994, -0.0246059 ,  0.05292112,
        0.12277183, -0.16477899, -0.16083917, -0.22634934, -0.1034404 ,
        0.10632364, -0.01822525, -0.0114993 ,  0.04524447, -0.23437706,
       -0.27861577, -0.07145165, -0.10128508,  0.0903349 , -0.14441672,
        0.01707909, -0.03193469, -0.1455247 , -0.07796918,  0.02380305,
        0.00708509, -0.04617143, -0.09996295,  0.16534936,  0.09301843,
       -0.20689608,  0.08410276, -0.00193592,  0.33988947,  0.29497528,
        0.06139373,  0.07164442, -0.06280392,  0.01383303, -0.26104403,
        0.04817868,  0.12485193,  0.13748586,  0.09956717,  0.09879835,
       -0.11403167,  0.00589237,  0.11198126, -0.19326156,  0.0511108 ,
        0.04213471, -0.04731822,  0.01416795, -0.10521804,  0.13353063,
        0.0422986 , -0.11264879, -0.09896166,  0.20074968, -0.20

## Face Comparison
To compare two faces, calculate the distance between their embeddings. A common metric is the Euclidean distance. If the distance is below a certain threshold (e.g., 0.6), the faces are likely to be of the same person.

In [7]:
from scipy.spatial import distance


max_dist = 1.0
idx = -1

# Calculate Euclidean distance between two encodings
for i, val in enumerate(encodings):
    encoding1, encoding2 = val
    dist = distance.euclidean(encoding1, encoding2)

    if dist < max_dist:
        max_dist = dist
        idx = i


# Define a threshold for determining if they are the same person
threshold = 0.6

print(max_dist)

if dist < threshold:
    print("The two images are likely of the same person.")
else:
    print("The two images are likely of different people.")

0.40255146180208334
The two images are likely of the same person.


## Sandbox

In [ ]:
def mse(imageA, imageB, is_resize: bool=False):
    # Ensure the images have the same size
    if imageA.shape != imageB.shape:
        if not is_resize:
            assert imageA.shape == imageB.shape, "Images must be the same size."
        imageB = cv2.resize(imageB, (imageA.shape[1], imageA.shape[0]))
    
    # Calculate the MSE between the images
    err = np.sum((imageA.astype("float") - imageB.astype("float")) ** 2)
    err /= float(imageA.shape[0] * imageA.shape[1])
    
    return err

# Load images
imageA = cv2.imread(IMG1, cv2.IMREAD_GRAYSCALE)
imageB = cv2.imread(IMG2, cv2.IMREAD_GRAYSCALE)

# Compute MSE
error = mse(imageA, imageB, is_resize=True)
print(f"Mean Squared Error: {error}")

Mean Squared Error: 7495.5450525480865


In [ ]:
from skimage.metrics import structural_similarity as ssim
import cv2

def compare_ssim(imageA, imageB, is_resize: bool=False):
    # Ensure the images have the same size
    if imageA.shape != imageB.shape:
        if not is_resize:
            assert imageA.shape == imageB.shape, "Images must be the same size."
        imageB = cv2.resize(imageB, (imageA.shape[1], imageA.shape[0]))
    
    # Compute SSIM between two images
    score, diff = ssim(imageA, imageB, full=True)
    return score

# Load images
imageA = cv2.imread(IMG1, cv2.IMREAD_GRAYSCALE)
imageB = cv2.imread(IMG2, cv2.IMREAD_GRAYSCALE)

# Compute SSIM
ssim_score = compare_ssim(imageA, imageB, is_resize=True)
print(f"SSIM Score: {ssim_score}")

SSIM Score: 0.0589125209112179


In [ ]:
import cv2

def compare_histograms(imageA, imageB, method='correlation'):
    # Convert images to HSV color space for better comparison
    imageA = cv2.cvtColor(imageA, cv2.COLOR_BGR2HSV)
    imageB = cv2.cvtColor(imageB, cv2.COLOR_BGR2HSV)

    # Calculate histograms
    histA = cv2.calcHist([imageA], [0, 1], None, [50, 60], [0, 180, 0, 256])
    histB = cv2.calcHist([imageB], [0, 1], None, [50, 60], [0, 180, 0, 256])

    # Normalize histograms
    cv2.normalize(histA, histA, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX)
    cv2.normalize(histB, histB, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX)

    # Use correlation or other methods
    methods = {
        'correlation': cv2.HISTCMP_CORREL,
        'chi-square': cv2.HISTCMP_CHISQR,
        'bhattacharyya': cv2.HISTCMP_BHATTACHARYYA
    }

    comparison = cv2.compareHist(histA, histB, methods[method])
    return comparison

# Load images
imageA = cv2.imread(IMG1)
imageB = cv2.imread(IMG2)

# Compare histograms using correlation
hist_score = compare_histograms(imageA, imageB, method='correlation')
print(f"Histogram Comparison Score (Correlation): {hist_score}")

Histogram Comparison Score (Correlation): 0.03436912205227033


In [ ]:
import cv2

def orb_feature_compare(imageA, imageB):
    # Convert to grayscale
    imageA = cv2.cvtColor(imageA, cv2.COLOR_BGR2GRAY)
    imageB = cv2.cvtColor(imageB, cv2.COLOR_BGR2GRAY)

    # Initialize ORB detector
    orb = cv2.ORB_create()

    # Find the keypoints and descriptors with ORB
    kpA, desA = orb.detectAndCompute(imageA, None)
    kpB, desB = orb.detectAndCompute(imageB, None)

    # Match descriptors
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
    matches = bf.match(desA, desB)
    
    # Sort matches by distance
    matches = sorted(matches, key=lambda x: x.distance)
    
    return len(matches), matches

# Load images
imageA = cv2.imread(IMG1)
imageB = cv2.imread(IMG2)

# Compare features using ORB
matches_count, matches = orb_feature_compare(imageA, imageB)
print(f"Number of Matches: {matches_count}")

Number of Matches: 127


In [ ]:
import torch
import torch.nn as nn
from torchvision import models, transforms
from PIL import Image

# Load pre-trained ResNet model
model = models.resnet50(pretrained=True)
model = nn.Sequential(*list(model.children())[:-1])  # Remove the last classification layer

def preprocess_image(image_path):
    preprocess = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    img = Image.open(image_path)
    img = preprocess(img).unsqueeze(0)  # Add batch dimension
    return img

def cosine_similarity(featureA, featureB):
    cos = nn.CosineSimilarity(dim=1, eps=1e-6)
    return cos(featureA, featureB)

# Load and preprocess images
imageA = preprocess_image(IMG1)
imageB = preprocess_image(IMG2)

# Extract features
with torch.no_grad():
    featuresA = model(imageA)
    featuresB = model(imageB)

# Compare using cosine similarity
similarity = cosine_similarity(featuresA, featuresB)
print(f"Cosine Similarity: {similarity.item()}")

ModuleNotFoundError: No module named 'torch'

In [ ]:
from transliterate import translit

# Original Russian text
russian_text = "Una niña"

# Transliterate to English
english_text = translit(russian_text, 'ru', reversed=True)
print(english_text)  # Output: Privet, kak dela?


Una niña


In [ ]:
text = 'aaa  aa.fffgf.gf'

a = text[:text.rfind('.')]
a = a.replace(' ', '_')
a

'aaa__aa.fffgf'

In [ ]:
from deep_translator import GoogleTranslator

# text = "Bonjour tout le monde"
text =  "Una niña"
text =  "Скриншот"
text =  "abFdAff"
text = "привет"

# Transliterate text
translated = GoogleTranslator(source='auto', target='en').translate(text)
print(translated)  # Output: Hello


Hello


In [ ]:
import cv2
# Load images

image1 = cv2.imread(IMG1)
image2 = cv2.imread(IMG2)

hist_img1 = cv2.calcHist([image1], [0, 1, 2], None, [256, 256, 256], [0, 256, 0, 256, 0, 256])
hist_img1[255, 255, 255] = 0 #ignore all white pixels
cv2.normalize(hist_img1, hist_img1, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX)
hist_img2 = cv2.calcHist([image2], [0, 1, 2], None, [256, 256, 256], [0, 256, 0, 256, 0, 256])
hist_img2[255, 255, 255] = 0  #ignore all white pixels
cv2.normalize(hist_img2, hist_img2, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX)
# Find the metric value
metric_val = cv2.compareHist(hist_img1, hist_img2, cv2.HISTCMP_CORREL)
print(f"Similarity Score: ", round(metric_val, 2))
# Similarity Score: 0.94

Similarity Score:  1.0


In [ ]:
import cv2
from skimage import metrics
# Load images
image1 = cv2.imread(IMG1)
image2 = cv2.imread(IMG2)
image2 = cv2.resize(image2, (image1.shape[1], image1.shape[0]), interpolation = cv2.INTER_AREA)
print(image1.shape, image2.shape)
# Convert images to grayscale
image1_gray = cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY)
image2_gray = cv2.cvtColor(image2, cv2.COLOR_BGR2GRAY)
# Calculate SSIM
ssim_score = metrics.structural_similarity(image1_gray, image2_gray, full=True)
print(f"SSIM Score: ", round(ssim_score[0], 2))
# SSIM Score: 0.38

(246, 205, 3) (246, 205, 3)
SSIM Score:  0.06


In [1]:
class SingletonClass:
    _instance = None

    def __new__(cls):
        if cls._instance is None:
            cls._instance = super(SingletonClass, cls).__new__(cls)
        return cls._instance

# Usage
singleton1 = SingletonClass()
singleton2 = SingletonClass()
print(singleton1 is singleton2)  # Output: True


True


In [2]:
def add_and_sort(distances, new_distance, new_id):
    # Append the new tuple (distance, id) to the list
    distances.append((new_distance, new_id))
    
    # Sort the list of tuples based on the first element (distance)
    distances.sort(key=lambda x: x[0])  # Sort by distance (first element of tuple)
    
    return distances

# Example usage
sorted_distances = []

# Simulating adding new elements
sorted_distances = add_and_sort(sorted_distances, 2.5, 'A')
sorted_distances = add_and_sort(sorted_distances, 1.2, 'B')
sorted_distances = add_and_sort(sorted_distances, 3.8, 'C')

print(sorted_distances)  # Output: [(1.2, 'B'), (2.5, 'A'), (3.8, 'C')]


[(1.2, 'B'), (2.5, 'A'), (3.8, 'C')]


In [5]:
import bisect


def add_sorted(distances, new_distance, new_id):
    # Create a tuple for the new element
    new_tuple = (new_distance, new_id)

    # Use bisect.insort to insert while maintaining order
    bisect.insort(distances, new_tuple)


# Example usage
sorted_distances = []

add_sorted(sorted_distances, 2.5, 'A')
add_sorted(sorted_distances, 1.2, 'B')
add_sorted(sorted_distances, 3.8, 'C')

print(sorted_distances)  # Output: [(1.2, 'B'), (2.5, 'A'), (3.8, 'C')]

[(1.2, 'B'), (2.5, 'A'), (3.8, 'C')]
